In [1]:
from pathlib import Path
import re
import requests
import json
from collections import defaultdict, Counter
from typing import List, Tuple, Literal
import numpy as np

In [2]:
# To get domain list file from an update, run in bash:
#     DATA="~/Workspace/Python/OverProt/data/multifamily"
#     python collect_domain_lists.py $DATA/all_unique_pdb-20210916/ $DATA/domain_list_20210916.csv

In [3]:
LIST_DIR = Path('/home/adam/Workspace/Python/OverProt/data/multifamily/domain_lists_OverProt_database')
RE_LIST_FILE = re.compile('domain_list_(.*)\.csv')
PDBEAPI_RELEASE_STATUS = 'https://www.ebi.ac.uk/pdbe/api/pdb/entry/status/{pdb}'

In [4]:
list_files = sorted(LIST_DIR.glob('domain_list_*.csv'))
# print(list_files)
lists = {}
dom2pdbchainranges = defaultdict(set)
for list_file in list_files:
    update_name = RE_LIST_FILE.match(list_file.name).group(1)
    print(update_name)
    the_list = []
    with open(list_file) as r:
        for line in r:
            if line.strip() != '':
                family, domain, pdb, chain, ranges = line.split()
                the_list.append((family, domain))
                dom2pdbchainranges[domain].add((pdb, chain, ranges))
    lists[update_name] = the_list

20201019
20201217
20210118
20210217
20210318
20210419
20210518
20210617
20210716
20210816
20210916


In [5]:
dom_to_fams = defaultdict(set)
for the_list in lists.values():
    for family, domain in the_list:
        dom_to_fams[domain].add(family)

In [6]:
# Does any domain belong to more than 1 family?
any(len(fams) != 1 for dom, fams in dom_to_fams.items())

False

In [7]:
update_names = list(lists.keys())
domain_sets = [set(dom for fam, dom in the_list) for the_list in lists.values()]
all_domains = set.union(*domain_sets)
static_domains = set.intersection(*domain_sets)
changing_domains = sorted(all_domains - static_domains)

print(f'All protein domains: {len(all_domains):6d}')
print(f'Static domains:      {len(static_domains):6d}')
print(f'Changing domains:    {len(changing_domains):6d}')

All protein domains: 471896
Static domains:      460482
Changing domains:     11414


In [8]:
print('Update    Domains  Missing')
for update_name, the_list in lists.items():
    print(f'{update_name}   {len(the_list):d}   {len(all_domains)-len(the_list):6d}')

Update    Domains  Missing
20201019   471896        0
20201217   464291     7605
20210118   469940     1956
20210217   471324      572
20210318   471322      574
20210419   471258      638
20210518   471293      603
20210617   471331      565
20210716   471326      570
20210816   471308      588
20210916   471284      612


In [9]:
presence_profiles = {domain: [] for domain in sorted(all_domains)}
for domain_set in domain_sets:
    for dom in all_domains:
        presence = int(dom in domain_set)
        presence_profiles[dom].append(presence)
presence_profiles = {domain: tuple(profile) for domain, profile in presence_profiles.items()}

In [10]:
def is_addition_profile(profile: Tuple[int]) -> bool:
    '''Decide if the profile has form [0]*m + [1]*n, where m, n >= 1, e.g. [0 0 0 1 1 1 1 1]'''
    if profile[0]:
        return False
    added = False
    for presence in profile:
        if not added and presence:
            added = True
        elif added and not presence:
            return False
    return added
    
def is_deletion_profile(profile: Tuple[int]) -> bool:
    '''Decide if the profile has form [1]*m + [0]*n, where m, n >= 1, e.g. [1 1 1 1 1 0 0 0]'''
    return is_addition_profile(invert_profile(profile))

def is_static_profile(profile: Tuple[int]) -> bool:
    '''Decide if the profile has form [1]*m, where m >= 1, e.g. [1 1 1 1 1 1 1 1]'''
    return all(profile)

def is_fluctuating_profile(profile: Tuple[int]) -> bool:
    '''Decide if the profile is fluctuating (i.e. not static or addition or deletion), e.g. [1 1 0 0 1 1 1 1]'''
    return not is_static_profile(profile) and not is_addition_profile(profile) and not is_deletion_profile(profile)

def profile_type(profile: Tuple[int]) -> Literal['static', 'addition', 'deletion', 'fluctuating']:
    if is_static_profile(profile): return 'static'
    elif is_addition_profile(profile): return 'addition'
    elif is_deletion_profile(profile): return 'deletion'
    else: return 'fluctuating'

def invert_profile(profile: Tuple[int]) -> Tuple[int]:
    return tuple(1-p for p in profile)

In [11]:
static_profiles = {domain: profile for domain, profile in presence_profiles.items() if is_static_profile(profile)}
addition_profiles = {domain: profile for domain, profile in presence_profiles.items() if is_addition_profile(profile)}
deletion_profiles = {domain: profile for domain, profile in presence_profiles.items() if is_deletion_profile(profile)}
fluctuating_profiles = {domain: profile for domain, profile in presence_profiles.items() if is_fluctuating_profile(profile)}

print(f'Static profiles:      {len(static_profiles):6d}')
print(f'Addition profiles:    {len(addition_profiles):6d}')
print(f'Deletion profiles:    {len(deletion_profiles):6d}')
print(f'Fluctuating profiles: {len(fluctuating_profiles):6d}')

Static profiles:      460482
Addition profiles:         0
Deletion profiles:       254
Fluctuating profiles:  11160


In [12]:
print(f'From all changing profiles:')
print(f'    Addition profiles:    {len(addition_profiles)/len(changing_domains):.2%}')
print(f'    Deletion profiles:    {len(deletion_profiles)/len(changing_domains):.2%}')
print(f'    Fluctuating profiles: {len(fluctuating_profiles)/len(changing_domains):.2%}')

From all changing profiles:
    Addition profiles:    0.00%
    Deletion profiles:    2.23%
    Fluctuating profiles: 97.77%


In [13]:
for typ, criterion in [('static', is_static_profile), ('added', is_addition_profile), ('deleted', is_deletion_profile), ('fluctuating', is_fluctuating_profile)]:
    with open(LIST_DIR/f'{typ}_domains.txt', 'w') as w:
        for domain, profile in presence_profiles.items():
            if criterion(profile):
                print(domain, file=w)

In [14]:
profile_counter = Counter(presence_profiles.values())
unique_profiles = profile_counter.keys()
for profile, count in profile_counter.most_common():
    print(np.array(profile), f'{count:6n}', f'({profile_type(profile)})' if not is_fluctuating_profile(profile) else '')

[1 1 1 1 1 1 1 1 1 1 1] 460482 (static)
[1 0 1 1 1 1 1 1 1 1 1]   7460 
[1 1 0 1 1 1 1 1 1 1 1]   1805 
[1 1 1 1 1 1 1 0 0 1 1]    359 
[1 1 1 0 0 1 1 1 1 0 1]    306 
[1 1 1 1 1 0 1 1 1 1 1]    261 
[1 1 1 1 1 1 0 1 1 1 0]    134 
[1 1 1 1 1 1 1 1 1 1 0]    129 (deletion)
[1 1 1 1 1 0 0 1 1 1 1]    115 
[1 1 1 1 1 1 0 1 1 1 1]    107 
[1 1 1 1 1 0 0 1 1 1 0]    101 
[1 1 1 0 0 1 1 1 1 1 1]     85 
[1 1 1 0 0 0 0 0 0 0 0]     45 (deletion)
[1 1 1 1 1 1 1 1 1 0 1]     40 
[1 1 0 0 0 1 1 1 1 0 1]     40 
[1 1 1 1 1 0 1 1 1 1 0]     36 
[1 1 0 1 1 1 1 0 0 1 1]     35 
[1 0 0 1 1 1 1 1 1 1 1]     34 
[1 1 1 1 1 1 0 0 0 0 0]     28 (deletion)
[1 0 1 1 1 1 1 1 1 1 0]     26 
[1 0 1 0 0 1 1 1 1 0 1]     25 
[1 1 0 0 0 1 1 1 1 1 1]     23 
[1 1 1 1 1 1 1 0 0 0 0]     22 (deletion)
[1 0 1 1 1 1 0 1 1 1 0]     20 
[1 1 1 1 1 0 1 0 0 1 1]     18 
[1 1 1 1 1 1 1 1 1 0 0]     16 (deletion)
[1 0 1 1 1 0 1 1 1 1 1]     15 
[1 0 1 1 1 1 1 0 0 1 1]     15 
[1 1 0 1 1 1 1 1 1 0 1]     10 
[1 1 1 0 0 0 1

In [15]:
for domain, profile in presence_profiles.items():
    if profile == (1, 1, 0, 1, 1, 1, 1, 0, 0, 1):
        print(domain, profile)

In [16]:
example_domain = '1vstA00'
print(example_domain, np.array(presence_profiles[example_domain]))

1vstA00 [1 1 0 1 1 1 1 0 0 1 1]


In [24]:
deleted_active_domains = []
deleted_obsolete_domains = []
for domain, profile in deletion_profiles.items():
    pdb = domain[:4]
    url = PDBEAPI_RELEASE_STATUS.format(pdb=pdb)
    response = requests.get(url).text
    js = json.loads(response)
    assert len(js[pdb]) == 1
    status = js[pdb][0]['status_code']
    if status == 'REL':
        deleted_active_domains.append(domain)
    elif status == 'OBS':
        deleted_obsolete_domains.append(domain)
    else:
        raise Exception(f'Unknown release status: {status}')
print(f'Deleted active domains:   {len(deleted_active_domains):6d}')
print(f'Deleted obsolete domains: {len(deleted_obsolete_domains):6d}')

In [25]:
for domain, profile in deletion_profiles.items():
    deleted_in_last = all(profile[:-1]) and not profile[-1]
    if not deleted_in_last and domain in deleted_active_domains:
        print(domain, end=', ')

In [ ]:
for domset in domain_sets:
    for domset2 in domain_sets:
        print(int(domset <= domset2), end='  ')
    print()

1  0  0  0  0  0  0  0  0  0  0  
1  1  0  0  0  0  0  0  0  0  0  
1  0  1  0  0  0  0  0  0  0  0  
1  0  0  1  0  0  0  0  0  0  0  
1  0  0  0  1  0  0  0  0  0  0  
1  0  0  0  0  1  0  0  0  0  0  
1  0  0  0  0  0  1  0  0  0  0  
1  0  0  0  0  0  0  1  0  0  0  
1  0  0  0  0  0  0  0  1  0  0  
1  0  0  0  0  0  0  0  0  1  0  
1  0  0  0  0  0  0  0  0  0  1  


In [18]:
dom_to_fam = {dom: next(iter(fams)) for dom, fams in dom_to_fams.items()}
messy_families = sorted(set(dom_to_fam[dom] for dom in fluctuating_profiles.keys()))
n_messy_families = len(messy_families)
n_families = len(set(dom_to_fam.values()))
print(f'Families with at least one fluctuating domain: {n_messy_families:6d}')
print(f'Total non-empty families in CATH:              {n_families:6d}')

Families with at least one fluctuating domain:   1008
Total non-empty families in CATH:                6567


In [ ]:
messy_immunoglobulins = [dom for dom, prof in fluctuating_profiles.items() if dom_to_fam[dom] == '2.60.40.10']
print(len(messy_immunoglobulins))
print(*messy_immunoglobulins)
for dom in messy_immunoglobulins:
    print(dom, np.array(presence_profiles[dom]))


143
1cg9B00 1d6vH01 1d6vL01 1f6aB01 1hxmF01 1hxyA02 1kfaM02 1owwA00 1ut9A01 1wwcA00 2d7qA01 2dijA03 2qkiE02 2x1pB00 2x1pD00 2x1wL01 2x2yA02 3bquD02 3c5jB02 3difD01 3ew3C01 3f65C02 3fqrA02 3k0wA01 3k0wA02 3piqG01 3qa3D01 3qazF01 3qazN02 3qazW01 3qazd02 3qpqE01 3rpiH01 3rpiL01 3utpD02 3utzA01 4bq6B02 4bq9A02 4bqbC01 4fqjL02 4fqlL01 4fxlA00 4hemE00 4k07A00 4k07B00 4k07C00 4k07D00 4k07E00 4k07F00 4k07G00 4k07H00 4k07I00 4k07J00 4kfzD00 4ki5F01 4ll9B01 4ll9B02 4lldA00 4m5yL01 4m5zH01 4mayD02 4n90E01 4n9gH02 4pjfF02 4pjhG02 4pjhH02 4pjiF01 4pjiG01 4pjxH02 4ut6I00 4ut9M00 4utaH01 4utaM01 4wu7A02 4ww1A01 4ww1A02 4ww1B01 4ww1B02 4ww2A01 4ww2A02 4ww2B01 4ww2B02 4ww2C02 4ww2F00 4wwiD01 4wwiD02 4wwiE01 4wwiE02 4wwiF01 4wwiF02 4wwkA01 4wwkA02 4wwkB01 4wwkB02 4wwkC02 4wwkD00 4xb8A04 4xbgA01 4xbpF02 4xpbL01 4y4fA02 4y4fC02 4y4fH01 4z5rT02 5c71C02 5cgyC01 5d7iD00 5d7iG01 5d7kD01 5d7lC02 5dfvC01 5f6hO02 5l0qB02 5utyL02 5v4mD02 5v4mJ02 5wwiA02 5wwiB00 5wwjA02 5wwjB00 5wwjC02 5wwjD00 5wwuA02 5wwuB00 5x08

In [ ]:
fluct_range_domains = []
chain_weirdos = []
ranges_weirdos = []
fluct_fragment_domains = []
for dom, pdbchainranges in dom2pdbchainranges.items():
    if len(pdbchainranges) != 1:
        fluct_range_domains.append(dom)
        # print(dom, *pdbchainranges)
        (pdb1, chain1, ranges1), (pdb2, chain2, ranges2) = pdbchainranges
        assert pdb1 == pdb2
        if chain1 == chain2:
            rangeset1 = set(ranges1.split(','))
            rangeset2 = set(ranges2.split(','))
            if rangeset1 < rangeset2 or rangeset2 < rangeset1:
                fluct_fragment_domains.append((dom, pdbchainranges))
            else:
                ranges_weirdos.append((dom, pdbchainranges))
            # if not (ranges1.startswith(ranges2) or ranges1.endswith(ranges2) or ranges2.startswith(ranges1) or ranges2.endswith(ranges1)):
            #     ranges_weirdos.append((dom, pdbchainranges))
        else:
            chain_weirdos.append((dom, pdbchainranges))

print('Domains with fluctuating ranges:', len(fluct_range_domains))
Counter(len(pdbchainranges) for pdbchainranges in dom2pdbchainranges.values())
print(f'Domains with whole fluctuating fragments [{len(fluct_fragment_domains)}]:', '...', sep='\n    ')
print(f'Weird cases (chain-wise) [{len(chain_weirdos)}]:', *chain_weirdos, sep='\n    ')
print(f'Weird cases (range-wise) [{len(ranges_weirdos)}]:', *ranges_weirdos, sep='\n    ')

Domains with fluctuating ranges: 1319
Domains with whole fluctuating fragments [1303]:
    ...
Weird cases (chain-wise) [10]:
    ('6nj9A00', {('6nj9', 'G', '37:135'), ('6nj9', 'A', '37:135')})
    ('6nj9B00', {('6nj9', 'J', '15:102'), ('6nj9', 'B', '15:102')})
    ('6nj9C00', {('6nj9', 'C', '11:119'), ('6nj9', 'I', '11:119')})
    ('6nj9D00', {('6nj9', 'H', '27:121'), ('6nj9', 'D', '27:121')})
    ('6nj9E00', {('6nj9', 'E', '37:135'), ('6nj9', 'F', '37:135')})
    ('6nj9F00', {('6nj9', 'A', '15:102'), ('6nj9', 'F', '15:102')})
    ('6nj9G00', {('6nj9', 'B', '11:119'), ('6nj9', 'G', '11:119')})
    ('6nj9H00', {('6nj9', 'C', '27:121'), ('6nj9', 'H', '27:121')})
    ('6nj9K01', {('6nj9', 'K', '4:126'), ('6nj9', 'D', '4:126')})
    ('6nj9K02', {('6nj9', 'D', '141:332'), ('6nj9', 'K', '141:332')})
Weird cases (range-wise) [6]:
    ('6d3oA00', {('6d3o', 'A', '7:102'), ('6d3o', 'A', '6:101')})
    ('6d3oB00', {('6d3o', 'B', '6:100'), ('6d3o', 'B', '7:101')})
    ('4zecA03', {('4zec', 'A', '

In [20]:
1653-1586

67

In [22]:
deleted_cyps = [dom for dom, prof in deletion_profiles.items() if dom_to_fam[dom] == '1.10.630.10']
print('Deleted CYPs', len(deleted_cyps))
print(*deleted_cyps)
for dom in deleted_cyps:
    print(dom, np.array(presence_profiles[dom]))

Deleted CYPs 0



In [19]:
messy_cyps = [dom for dom, prof in fluctuating_profiles.items() if dom_to_fam[dom] == '1.10.630.10']
print('Fluctuating CYPs:', len(messy_cyps))
print(*messy_cyps)
for dom in messy_cyps:
    print(dom, np.array(presence_profiles[dom]))

52
1fahB00 1izoA00 1j51D00 1jpzB00 1p0xB00 1uedB00 1yqpA00 1z11D00 2a1oA00 2fdvD00 2iagA00 2p85C00 2qbnA00 2rfbA00 2rfcC00 2wx2A00 2x2nC00 2ycaA00 3g5fA00 3l62A00 3lc4B00 3mzsB00 3nc3B00 3nplA00 3psxB00 3tmzA00 3w9cA00 3wrkA00 3zbyA00 3zbyD00 4g7gC00 4k9vA00 4kewA00 4l0eA00 4l4cA00 4nkxC00 4nkzD00 4tpnA00 4triA00 4zdzA01 4zgxE00 5de9B00 5e58E00 5jq2A00 5jquC00 5jquD00 5jquE00 5kyoC00 5veuK00 5yhjB00 6dwmD00 6fyjA00
1fahB00 [1 1 1 1 1 0 0 1 1 1 0]
1izoA00 [1 1 1 1 1 0 0 1 1 1 0]
1j51D00 [1 1 1 1 1 0 0 1 1 1 0]
1jpzB00 [1 1 1 1 1 0 0 1 1 1 0]
1p0xB00 [1 1 1 1 1 0 0 1 1 1 0]
1uedB00 [1 1 1 1 1 0 0 1 1 1 0]
1yqpA00 [1 1 1 0 0 0 0 1 1 0 0]
1z11D00 [1 1 1 1 1 0 0 1 1 1 0]
2a1oA00 [1 1 1 1 1 0 0 1 1 1 0]
2fdvD00 [1 1 1 1 1 0 0 1 1 1 0]
2iagA00 [1 1 1 1 1 0 0 1 1 1 0]
2p85C00 [1 1 1 1 1 0 0 1 1 1 0]
2qbnA00 [1 1 1 1 1 0 0 1 1 1 0]
2rfbA00 [1 1 1 1 1 0 0 0 0 1 0]
2rfcC00 [1 1 1 1 1 0 0 0 0 1 0]
2wx2A00 [1 1 1 1 1 0 0 1 1 1 0]
2x2nC00 [1 1 1 0 0 0 0 1 1 1 0]
2ycaA00 [1 1 1 1 1 0 0 1 1 1 0]
3g5fA